# Problématique métier

Le but est de prédire la condition opérationel du "point d'eau".

Les status possibles sont les suivants : 
* functional
* functional needs repair
* non functional

# Préparation des données

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
#pd.set_option('max_columns', None)

In [2]:
raw_data = pd.read_csv("input/features.csv", ",")

D:\ESGI\5AL\ML\ML-TP\venv\lib\site-packages\IPython\core\interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Regarder dans la base
raw_data.head() # afficher les 5 premières lignes

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
# Vérification si chaque lignes à bien un id
len(raw_data["id"].unique())

59400

In [5]:
print(raw_data.shape)
raw_data.info()

(59400, 40)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int

## Recherche des valeurs manquantes

In [6]:
# nombre de valeurs manquantes
print(raw_data.isnull().sum().sort_values(ascending=False))

scheme_name              28166
scheme_management         3877
installer                 3655
funder                    3635
public_meeting            3334
permit                    3056
subvillage                 371
management_group             0
waterpoint_type              0
extraction_type              0
extraction_type_group        0
source_class                 0
extraction_type_class        0
management                   0
source_type                  0
source                       0
payment_type                 0
water_quality                0
construction_year            0
quality_group                0
quantity                     0
quantity_group               0
payment                      0
id                           0
amount_tsh                   0
recorded_by                  0
population                   0
ward                         0
lga                          0
district_code                0
region_code                  0
region                       0
basin   

Après analyse, nous allons enlever le scheme_name

In [7]:
# Proportion de valeurs manquantes
print(raw_data.isnull().mean().sort_values(ascending=False))

scheme_name              0.474175
scheme_management        0.065269
installer                0.061532
funder                   0.061195
public_meeting           0.056128
permit                   0.051448
subvillage               0.006246
management_group         0.000000
waterpoint_type          0.000000
extraction_type          0.000000
extraction_type_group    0.000000
source_class             0.000000
extraction_type_class    0.000000
management               0.000000
source_type              0.000000
source                   0.000000
payment_type             0.000000
water_quality            0.000000
construction_year        0.000000
quality_group            0.000000
quantity                 0.000000
quantity_group           0.000000
payment                  0.000000
id                       0.000000
amount_tsh               0.000000
recorded_by              0.000000
population               0.000000
ward                     0.000000
lga                      0.000000
district_code 

## Préparation des données

In [8]:
string_list = ["funder", "installer", "wpt_name", "basin", "subvillage", "region", "lga", "ward", "recorded_by", "scheme_management", "scheme_name", "extraction_type", "extraction_type_group", "extraction_type_class", "management", "management_group", "payment", "payment_type", "water_quality", "quality_group", "quantity", "quantity_group", "source", "source_type", "source_class", "waterpoint_type", "waterpoint_type_group"]

In [9]:
raw_data_conv = raw_data.copy()
#a mettre avant
raw_data_conv[string_list] = raw_data_conv[string_list].astype(str)
raw_data_conv[string_list] = raw_data_conv[string_list].astype("string")
raw_data_conv[string_list] = raw_data_conv[string_list].replace({pd.NA: np.nan})
raw_data_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  string 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  string 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  string 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  string 
 11  subvillage             59400 non-null  string 
 12  region                 59400 non-null  string 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Convervion des dates

In [10]:
raw_data_conv['date_recorded'] =  pd.to_datetime(raw_data_conv['date_recorded'], format='%Y-%m-%d')

In [11]:
raw_data_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     59400 non-null  int64         
 1   amount_tsh             59400 non-null  float64       
 2   date_recorded          59400 non-null  datetime64[ns]
 3   funder                 59400 non-null  string        
 4   gps_height             59400 non-null  int64         
 5   installer              59400 non-null  string        
 6   longitude              59400 non-null  float64       
 7   latitude               59400 non-null  float64       
 8   wpt_name               59400 non-null  string        
 9   num_private            59400 non-null  int64         
 10  basin                  59400 non-null  string        
 11  subvillage             59400 non-null  string        
 12  region                 59400 non-null  string        
 13  r

Convertion des colonnes "object" restantes en boolean

In [12]:
# récupérer les colonnes à convertir
objectcols = raw_data_conv.select_dtypes(include='object').columns
print(objectcols)

Index(['public_meeting', 'permit'], dtype='object')


In [13]:
# Convertion des données en boolean
raw_data_conv[objectcols] = raw_data_conv[objectcols].astype(bool)
raw_data_conv[objectcols] = raw_data_conv[objectcols].astype("bool")
raw_data_conv[objectcols] = raw_data_conv[objectcols].replace({pd.NA: np.nan})

In [14]:
raw_data_conv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     59400 non-null  int64         
 1   amount_tsh             59400 non-null  float64       
 2   date_recorded          59400 non-null  datetime64[ns]
 3   funder                 59400 non-null  string        
 4   gps_height             59400 non-null  int64         
 5   installer              59400 non-null  string        
 6   longitude              59400 non-null  float64       
 7   latitude               59400 non-null  float64       
 8   wpt_name               59400 non-null  string        
 9   num_private            59400 non-null  int64         
 10  basin                  59400 non-null  string        
 11  subvillage             59400 non-null  string        
 12  region                 59400 non-null  string        
 13  r

## Préparation de la target

In [15]:
target = pd.read_csv("input/target.csv", delimiter=",")
y = target.drop("id", axis=1)
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  object
dtypes: object(1)
memory usage: 464.2+ KB


In [16]:
# Convertion du "status_group" en string
col = "status_group"
y[col] = y[col].astype(str)
y[col] = y[col].astype("string")
y[col] = y[col].replace({pd.NA: np.nan})
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  59400 non-null  string
dtypes: string(1)
memory usage: 464.2 KB


# Imputation & Sélection des variables explicatives

### Imputation

Après analyse des valeures manquantes : 
* scheme_name              28166 => à supprimer car plus de 50 % des datas
* scheme_management         3877 => mode
* installer                 3655 => mode
* funder                    3635 => mode
* public_meeting            3334 => mode
* permit                    3056 => mode
* subvillage                 371 => mode

In [17]:
from sklearn.impute import SimpleImputer

colstomode = ["scheme_management", "installer", "funder", "public_meeting", "permit", "subvillage"]
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit(raw_data_conv[colstomode])

# mean, most_frequent, median

data_impute = imputer.transform(raw_data_conv[colstomode].values)

data_impute = pd.DataFrame(data_impute, columns = colstomode)

D:\ESGI\5AL\ML\ML-TP\venv\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


In [18]:
raw_data_conv = raw_data_conv.drop(['scheme_name'], axis=1)

In [19]:
raw_data_conv = raw_data_conv.dropna(axis=1, how='all')
raw_data_conv.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


### Sélection des variables

In [20]:
# Suppression des variables : id, date_recorded
data_clean = raw_data_conv.drop(['id', 'date_recorded'], axis=1)

In [21]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   funder                 59400 non-null  string 
 2   gps_height             59400 non-null  int64  
 3   installer              59400 non-null  string 
 4   longitude              59400 non-null  float64
 5   latitude               59400 non-null  float64
 6   wpt_name               59400 non-null  string 
 7   num_private            59400 non-null  int64  
 8   basin                  59400 non-null  string 
 9   subvillage             59400 non-null  string 
 10  region                 59400 non-null  string 
 11  region_code            59400 non-null  int64  
 12  district_code          59400 non-null  int64  
 13  lga                    59400 non-null  string 
 14  ward                   59400 non-null  string 
 15  po

# Création des models

## Splitting des données

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data_clean, y, test_size=0.2, random_state=5)

print('Train set:', X_train.shape)
print('Test set:', X_test.shape)

Train set: (47520, 37)
Test set: (11880, 37)


## KNN

### Création

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
# trouver le k optimal pour le nb de neighbors avant la declaration du knn ci dessous 

## Automatique
# définissons les paramètress
param_grid = {'n_neighbors': np.arange(1, 20),
              'metric': ['euclidean', 'manhattan']}
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_knn.fit(X_train, y_train)
#print(grid_knn.best_score_)
print(grid_knn.best_params_)


from sklearn.neighbors import KNeighborsClassifier

# déclaration de l'intention de knn
knn = KNeighborsClassifier(n_neighbors=grid_knn.best_params_["n_neighbors"])

# Entrainement du modèle
knn.fit(X_train, y_train)

### Evaluation

In [ ]:
# Evaluation --------------

# Simple

#print('train score:', knn.score(X_train, y_train))
#print('test score:', knn.score(X_test, y_test))

from sklearn.metrics import f1_score, confusion_matrix, classification_report

y_test_predit = knn.predict(X_test)

confusion_matrix(y_test, y_test_predit)

# calcul de la précision, du rappel et du F-score
#print(classification_report(y_test, y_test_predit))


# Cross validation
from sklearn.model_selection import cross_val_score
cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy').mean()

# Courbe d'apprentissage
from sklearn.model_selection import learning_curve

N, train_score, val_score = learning_curve(knn, X_train, y_train, train_sizes=np.linspace(0.1, 1, 10), cv=5)
plt.plot(N, train_score.mean(axis=1), label='train')
plt.plot(N, val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()

## Random Forest

### Création

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
# Entrainement du modèle
rf.fit(X_train, y_train)

### Evaluation

In [ ]:
# Calcul de score

print('train score:', rf.score(X_train, y_train))
print('test score:', rf.score(X_test, y_test))

y_test_rf_predit = rf.predict(X_test)

N_rf, train_score_rf, val_score_rf = learning_curve(rf, X_train, y_train, 
                                                    train_sizes=np.linspace(0.1, 1, 10),
                                                    cv=5, scoring='accuracy')

confusion_matrix(y_test, y_test_rf_predit)

plt.plot(N_rf, train_score_rf.mean(axis=1), label='train_rf')
plt.plot(N_rf, val_score_rf.mean(axis=1), label='validation_rf')
plt.xlabel('train_sizes')
plt.legend()

# calcul de la précision, du rappel et du F-score
print(classification_report(y_test, y_test_rf_predit))

cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy').mean()
pd.DataFrame(rf.feature_importances_).plot.bar(figsize=(12, 8))

## Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
print('train score:', dt.score(X_train, y_train))
print('test score:', dt.score(X_test, y_test))
y_test_dt_predit = dt.predict(X_test)

#confusion_matrix(y_test, y_test_dt_predit)

#print(classification_report(y_test, y_test_dt_predit))
from sklearn.tree import export_graphviz
FEATURE_NAMES = ["age", "sex", "cp", "trestbps", "chol", "fbs", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]
#export_graphviz(dt, 'tree.dot', feature_names = FEATURE_NAMES)
!dot -Tpng tree.dot -o tree.png
import matplotlib.pyplot as plt
#import cv2 #pip install opencv-python opencv-python-headless
#%matplotlib inline
#img = cv2.imread('tree.png')
#plt.figure(figsize = (20, 20))
#plt.imshow(img)

## Regression linéaire

### Création

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=3)
X_train_poly = polynomial_features.fit_transform(X_train)
X_test_poly = polynomial_features.fit_transform(X_test)

# Entrainement
linreg = LinearRegression()
linreg.fit(X_train_poly, y_train)

### Evaluation

In [ ]:
# Evaluation
y_train_predict = linreg.predict(X_train_poly)
y_test_predict = linreg.predict(X_test_poly)
mse_train = mean_squared_error(y_train, y_train_predict)
rmse_train = (np.sqrt(mse_train))
r2_train = r2_score(y_train, y_train_predict)
mse_test = mean_squared_error(y_test, y_test_predict)
rmse_test = (np.sqrt(mse_test))
r2_test = r2_score(y_test, y_test_predict)
print('La performance du modèle sur la base dapprentissage')
print('--------------------------------------')
print('mse du train {}'.format(mse_train), ' et test {}'.format(mse_test))
print("L'erreur quadratique moyenne du train est {}".format(rmse_train), " et test est {}".format(rmse_test))
print('le score R2 du train est {}'.format(r2_train), ' et test est {}'.format(r2_test))
print('\n')

# Conclusion